# 11.3 Web Scraping

**HTML**, which stands for "hypertext markup language", is an XML-like language for specifying the appearance of web pages. Each tag in HTML corresponds to a specific page element. For example:

- `<img>` specifies an image. The path to the image file is specified in the `src=` attribute.
- `<a>` specifies a hyperlink. The text enclosed between `<a>` and `</a>` is the text of the link that appears, while the URL is specified in the `href=` attribute of the tag.
- `<table>` specifies a table. The rows of the table are specified by `<tr>` tags nested inside the `<table>` tag, while the cells in each row are specified by `<td>` tags nested inside each `<tr>` tag.

Our goal in this section is not to teach you HTML to make a web page. You will learn just enough HTML to be able to scrape data programmatically from a web page.

# Inspecting HTML Source Code

Suppose we want to scrape faculty information from the [Cal Poly Statistics Department directory](https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours) (`https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours`). Once we have identified a web page that we want to scrape, the next step is to study the HTML source code. All web browsers have a "View Source" or "Page Source" feature that will display the HTML source of a web page. 

Visit the web page above, and view the HTML source of that page. (You may have to search online to figure out how to view the page source in your favorite browser.) Scroll down until you find the HTML code for the table containing information about the name, office, phone, e-mail, and office hours of the faculty members.

Notice how difficult it can be to find a page element in the HTML source. Many browsers allow you to right-click on a page element and jump to the part of the HTML source corresponding to that element.

# Web Scraping Using BeautifulSoup

`BeautifulSoup` is a Python library that makes it easy to navigate an HTML document. Like with `lxml`, we can query tags by name or attribute, and we can narrow our search to the ancestors and descendants of specific tags. In fact, it is possible to use `lxml` with HTML documents, but many web sites have malformed HTML, and `lxml` is not very forgiving. `BeautifulSoup` handles malformed HTML more gracefully and is thus the library of choice.

First, we issue an HTTP request to the URL to get the HTML source code.

In [1]:
import requests
resp = requests.get("https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours")

The HTML source is stored in the `.content` attribute of the response object. We pass this HTML source into `BeautifulSoup` to obtain a tree-like representation of the HTML document.

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(resp.content, "html.parser")

Now we can search for tags within this HTML document, using tags like `.find_all()`. For example, we can find all tables on this page.

In [3]:
tables = soup.find_all("table")
len(tables)

2

As a visual inspection of [the web page](https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours) would confirm, there are 2 tables on the page, and we are interested in the second one.

In [4]:
table = tables[1]
table

<table cellspacing="0" class="directory backwhite" summary="COSAM Dean's Office Staff" width="100%"><tbody><tr><th scope="col" width="18%">
<p>Name</p>
</th>
<th scope="col" width="8%">Office</th>
<th scope="col" width="10%">Phone (805)</th>
<th scope="col" width="20%">Email</th>
<th scope="col" width="20%">
<p>WINTER 2019</p>
<p>Office Hours</p>
</th>
</tr><tr><td><a href="/Prince-Afriyie"><strong>Prince Afriyie</strong></a></td>
<td><a href="https://maps.calpoly.edu/">25-103</a></td>
<td>756-2715</td>
<td><a href="mailto:pafriyie@calpoly.edu">pafriyie@calpoly.edu</a></td>
<td>M-R: 2:10-3:00pm</td>
</tr><tr><td><a href="/Kelly-Bodwin"><strong>Kelly Bodwin</strong></a></td>
<td><a href="https://maps.calpoly.edu/">25-106</a></td>
<td>756-2450</td>
<td><a href="mailto:kbodwin@calpoly.edu">kbodwin@calpoly.edu</a></td>
<td>
<p>M/W: 12:30-1:30pm</p>
<p>T/R: 10:00-11:00am</p>
</td>
</tr><tr><td><a href="/Matt-Carlton"><strong>Matt Carlton</strong></a></td>
<td><a href="https://maps.calpoly.e

There is one faculty member per row, except for the first row, which is the header. We iterate over all rows except for the first, extracting information about each faculty to append to `rows`, which we will eventually turn into a `DataFrame`. As you read the code below, refer to the HTML source above, so that you understand what each line is doing. In particular, ask yourself the following questions:

- `cells[0]` represents a `<td>` tag. Why do we need to call `.find("strong")` within this tag to get the name of the faculty member?
- For the most part, `link` is a hyperlink whose text is the faculty's office number. But for some faculty, `link` is `None`. For which faculty is this the case and why?

You are encouraged to add `print()` statements inside the `for` loop to check your understanding of each line of code.

In [6]:
rows = []
for faculty in table.find_all("tr")[1:]:
    # Get all the cells in the row.
    cells = faculty.find_all("td")
    
    # The information we need is the text between tags.
    name = cells[0].find("strong").text
    
    link = cells[1].find("a")
    office = cells[1].text if link is None else link.text
    
    email = cells[3].find("a").text
    
    # Append this data.
    rows.append({
        "name": name,
        "office": office,
        "email": email
    })

In the code above, observe that `.find_all()` returns a list with all matching tags, while `.find()` returns only the first matching tag. If no matching tags are found, then `.find_all()` will return an empty list `[]`, while `.find()` will return `None`.

Finally, we turn `rows` into a `DataFrame`.

In [7]:
import pandas as pd
pd.DataFrame(rows)

,email,name,office
0,pafriyie@calpoly.edu,Prince Afriyie,25-103
1,kbodwin@calpoly.edu,Kelly Bodwin,25-106
2,mcarlton@calpoly.edu,Matt Carlton,25-101
3,bchance@calpoly.edu,Beth Chance,25-235
4,ncheng@calpoly.edu,Nianpin Cheng,192-232
5,clancy@calpoly.edu,Kevin Clancy,25-122
6,odekhtya@calpoly.edu,Olga Dekhtyar,25-225
7,jdoi@calpoly.edu,Jimmy Doi,25-108
8,sframe@calpoly.edu,Samuel Frame,25-121
9,hglanz@calpoly.edu,Hunter Glanz,25-111


Now this data is ready for further processing.

# Ethical Enlightenment: `robots.txt`

Web robots are crawling the web all the time. A website may want to restrict the robots from crawling specific pages. One reason is financial: each visit to a web page, by a human or a robot, costs the website money, and the website may prefer to save their limited bandwidth for human visitors. Another reason is privacy: a website may not want a search engine to preserve a snapshot of a page for all eternity.

To specify what a web robot is and isn't allowed to crawl, most websites will place a text file named `robots.txt` in the top-level directory of the web server. For example, the Statistics department web page has a `robots.txt` file: https://statistics.calpoly.edu/robots.txt

The format of the `robots.txt` file should be self-explanatory, but you can read a full specification of the standard here: http://www.robotstxt.org/robotstxt.html. As you scrape websites using your web robot, always check the `robots.txt` file first, to make sure that you are respecting the wishes of the website owner.

# Exercises

**Exercise 1.** The [Cal Poly course catalog](http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory) (`http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory`) contains a list of courses offered by the Statistics department. Scrape this website to obtain a `DataFrame`, where the unit of observation is a course. Use this `DataFrame` to answer the following questions: 

- How many 300-level courses does the Statistics department offer?
- How many distinct courses are offered in Spring quarter?

In [9]:
resp = requests.get("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
soup = BeautifulSoup(resp.content, "html.parser")

In [202]:
import re

course_df = pd.DataFrame()
courses = soup.find_all("div", class_="courseblock")
for course in courses:
    courseblock = course.p
    coursewrap = course.div
    
    pat_num = re.compile("(\d{3})")
    pat_term = re.compile("Offered:.(.*)")
    terms = [p.text for p in course.div.find_all("p") if "Term Typically Offered: " in p.text][0].replace(",", " ")
    course_df = course_df.append({"num": pat_num.search(course.text).group(0), "term": pat_term.search(terms).groups()[0].split()}, ignore_index=True)

In [203]:
course_df

,num,term
0,301,[W]
1,401,[F]
2,451,[W]
3,452,[SP]
4,130,"[F, W, SP]"
5,150,[F]
6,200,"[F, W, SP]"
7,217,"[F, W, SP, SU]"
8,218,"[F, W, SP, SU]"
9,251,"[F, W, SP]"


In [87]:
pd.Series(course_nums)

0     301
1     401
2     451
3     452
4     130
5     150
6     200
7     217
8     218
9     251
10    252
11    270
12    301
13    302
14    305
15    312
16    313
17    321
18    323
19    324
20    330
21    331
22    334
23    350
24    365
25    400
26    405
27    410
28    414
29    415
30    416
31    417
32    418
33    419
34    421
35    423
36    425
37    426
38    427
39    434
40    440
41    441
42    461
43    462
44    465
45    470
46    485
47    495
48    511
49    513
50    523
51    524
52    530
53    531
54    542
55    570
dtype: object